In [ ]:
%matplotlib inline
%env TQDM_DISABLE=1
import warnings
warnings.filterwarnings('ignore')
# tag: remove-cell applied

# Customize an Excel Export

After executing a batch analysis, you may want to add some additional information to the default Excel exports. The `pybmds` package stores all modeling information in a data structure that allows us to get both the data and reports. This notebook demonstrates running a simple batch analysis, and then augmenting the default Excel export with a few extra columns of information.

In [ ]:
from pprint import pprint

import pandas as pd

import pybmds

As a simple example, we'll generate a batch analysis using a few option sets and a single dataset. You could adapt this code to run a custom analysis of your choosing:

In [ ]:
def build_cont_sess(ds):
    def add_model(sess, Model, base, additions=None):
        settings = {
            "priors": pybmds.PriorClass.frequentist_restricted,
            "bmr_type": base[0],
            "bmr": base[1],
            "disttype": base[2],
        }
        if additions is not None:
            settings.update(additions)
        sess.add_model(Model, settings)

    option_sets = [
        (pybmds.ContinuousRiskType.RelativeDeviation, 0.1, pybmds.ContinuousDistType.normal),
        (pybmds.ContinuousRiskType.RelativeDeviation, 0.1, pybmds.ContinuousDistType.normal_ncv),
    ]
    sessions = []
    for option_set in option_sets:
        sess = pybmds.Session(dataset=ds)
        add_model(sess, pybmds.Models.ExponentialM3, option_set)
        add_model(sess, pybmds.Models.ExponentialM5, option_set)
        add_model(sess, pybmds.Models.Power, option_set)
        add_model(sess, pybmds.Models.Power, option_set)
        add_model(sess, pybmds.Models.Linear, option_set)
        add_model(sess, pybmds.Models.Polynomial, option_set, {"degree": 2})
        add_model(sess, pybmds.Models.Polynomial, option_set, {"degree": 3})
        sess.execute_and_recommend()
        sessions.append(sess.to_dict())
    return pybmds.BatchResponse(success=True, content=sessions)


datasets = [
    pybmds.ContinuousDataset(
        doses=[0, 10, 50, 150, 400],
        ns=[111, 142, 143, 93, 42],
        means=[2.112, 2.095, 1.956, 1.587, 1.254],
        stdevs=[0.235, 0.209, 0.231, 0.263, 0.159],
    )
]
sess_batch = pybmds.BatchSession.execute(datasets, build_cont_sess, nprocs=1)

To generate a standard Excel export, you'd call this method:

In [ ]:
sess_batch.to_excel('output/batch.xlsx')

However, we'll want to customize this export to add more information. In this example, we may want to show more information regarding Analysis of Deviance Table than is generally shown in the summary exports. First, let's generate the the summary dataframe, which we'll want to add more info to:

In [ ]:
df_summary = sess_batch.df_summary()
df_summary.head()

Which has 5 rows and 37 columns. Now, let's add some additional information to the summary table. We'll iterate all the sessions in our dataset, and all the models in each session, and create a new data frame that we can merge with the default summary data frame:

In [ ]:
rows = []
for i, session in enumerate(sess_batch.session):
    for j, model in enumerate(session.models):
        data = {
            "session_index": i,
            "bmds_model_index": j,
        }
        if model.has_results:
            res = model.results
            data.update({
                "A1_ll": res.deviance.loglikelihoods[0],
                "A2_ll": res.deviance.loglikelihoods[1],
                "A3_ll": res.deviance.loglikelihoods[2],
                "fitted_ll": res.deviance.loglikelihoods[3],
                "reduced_ll": res.deviance.loglikelihoods[4],
                "A1_aic": res.deviance.aics[0],
                "A2_aic": res.deviance.aics[1],
                "A3_aic": res.deviance.aics[2],
                "fitted_aic": res.deviance.aics[3],
                "reduced_aic": res.deviance.aics[4],
            })
        rows.append(data)

df2 = pd.DataFrame(rows)
df2.head()

Now, we can join the two data frames together, using the session and model keys to join:

In [ ]:
df_summary2 = pd.merge(df_summary, df2, on=["session_index", "bmds_model_index"])
df_summary2.head()

Now, the summary dataframe has 47 columns instead of 35.

Finally, we'll write the Excel export, with multiple tabs for different types of data:

In [ ]:
with pd.ExcelWriter('output/report.xlsx') as writer:
    data = {
        "summary": df_summary2,
        "datasets": sess_batch.df_dataset(),
        "parameters": sess_batch.df_params(),
    }
    for name, df in data.items():
        df.to_excel(writer, sheet_name=name, index=False)

This export includes our custom values!

## Model result introspection

A quick dive into introspecting the model results that are available and their data structures and text summaries.

Let's grab the first model that was executed and look at it's results object:

In [ ]:
model = sess_batch.session[0].models[0]
res = model.results

This is a nested python data structure, for example:

In [ ]:
print(f"{res.bmd=}")
print(f"{res.bmdl=}")
print(f"{res.fit.aic=}")

To better understand the structure, we can "pretty print" a dictionary representation:

In [ ]:
data = res.model_dump()

# truncate a few fields so they print better... (you can ignore this code)
data['fit']['bmd_dist'][0] = data['fit']['bmd_dist'][0][:5]
data['fit']['bmd_dist'][1] = data['fit']['bmd_dist'][1][:5]
data['plotting']['dr_x'] = data['plotting']['dr_x'][:5]
data['plotting']['dr_y'] = data['plotting']['dr_y'][:5]

pprint(data, indent=2, width=140, sort_dicts=True)

This may be helpful in trying to find particular values in the nested results.

### Build a text table

This is a helpful pattern to print data in a tabular format:

In [ ]:
for name, df, ll, p_value in zip(res.tests.names, res.tests.dfs, res.tests.ll_ratios, res.tests.p_values, strict=True):
    print(f"{name:10} {df: <6} {ll: <10.4f} {p_value: <8.6f}")